In [1]:
import re
import numpy as np
from collections import Counter
from math import inf
from pprint import pprint

# 고전 암호 문제풀이

Hill 암호를 평문 하나만으로 풀어야 한다. 우선 암호문을 `encrypted.txt`에 저장하였다.

In [2]:
with open('encrypted.txt') as f:
    encrypted = f.read().strip()
    print(encrypted)
print('length: %d' % len(encrypted))

HRDKHUBHAAMAEQMTMZSHGBAKFUBHAASYRXUNKYUAATQCTLUTOGEWVAJGVEIIYTKIOTQRXXQVSQLISVVOCNGCUXPKPIUBOHTVKCFKWNJSEZYSSUTUOESIXKAPVFXNZHAOQTLCGYJVAEHLNNKEESQMKSHKKDFCNZSRHRDKHSDKFXVPTGMKRUPZBIKEVNYEKXMFXKFYMWYUDZDENEWNKDAOUXGPCXZDLCSNFGCMCSNUAOJDBLQTAHEWYZCHQJYKSNUWOKQKONZGOKDXGUXKEMWQMCFGUEAVKHDIIATCHVTGYMGKJMLNPCNAYKMIRWEETIYQKELEGLQOVKISFNUDAJQIQYBXQTMZSHGBAKFZRCNWRSODAFKKXWGAZGDBIUDDHCUDFRFOVSZXADSHYSGLTQBMNEMKDCFSOZSRDYLIHIAXCMGMFEIDNZKOVJEOIEFNWWQEDRLZYZIZXADSHYSGLJYFWDUAKSIOGOZOXWYPBUFEPNBIRJUJNDZJJYMURKNCIKPWLRMRIAGVSXTYNIWPROHLDHQOMBEKZURQCLQOVKISFNUAFQBHGPCLHZTPJVPXIZKLQSNVKIJAEITTNVSVWNFYVATDEMKDCTGIHKZTVGZYXTYQEDBACFMNCAHRDKHSDKFXZXXGMJOSLPSZBMOILMMWRALAFFMNXXDYFBIYQVVOHSWKGBIRJGTBYQLKIJAEQBTAXGFGAVUIJADHQKLFWRJXYFVIGGQZNBHSUIYOZALSKIABLWQNXNXKOAJAIKHXODXWORVDOGBMHOPLQJZALQJZALIKTKLENZHQAVYUEUFEVLUXHGOWNMGWXUIAHGQOMNCKFQLIPBNKVWDLNGMJCOBFKIGBYWPAHMMPQLUTOGECXITZVVAJEOIDCNWMFNLOBGQXCYFWQFWVXWRKWYGBFHJVLBAWBOUQEKHZHSZZIZARYITDCLQFPGBTJMQVSQLIHPEJONCYMZWTVJVZOBOMOHPSXMPUKVAGXIPOQUQUQBCK

평문의 길이는 1285자이고, `1285 = 5 * 257`이다. 즉 blocksize는 5 또는 257이다.

정말 무식하게 시도하면 얻을 수 있는 시간복잡도는 `O(d^3*26^d^2)` 정도이다. 이 암호에 COA를 적용하기 위해 [찾아본 결과](https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q=ciphertext-only+attack+on+hill&btnG=&oq=hil), [분할 정복을 이용해 `O(26d^2)`까지 시간복잡도를 줄이는 논문](https://eprint.iacr.org/2015/802.pdf)이 있었고, 이를 이용해 보자. (사실 이 논문에서는 CRT를 이용해 무려 `O(d*13^d)`까지 줄였지만, 여기에서는 그정도로 빠른 것은 필요하지 않을 것으로 생각된다. ~~하지만 blocksize가 257이라면?~~)

논문의 의사 코드를 그대로 구현하여 보자. (함수 내부의 변수 이름은 논문의 것을 따라갔다.) 우선 주어진 텍스트에 있는 문자들의 normalized frequency를 계산하는 함수 `normalized_freq`을 구현한다.

In [3]:
def normalized_freq(text):
    counter = Counter(text)
    result = np.zeros((26,))
    for i in range(26):
        if i not in counter:
            result[i] = 0
            continue
        result[i] = counter[i]
    return result / len(text)

# some code for testing
text = [0, 1, 2, 4, 5, 3, 2, 2, 1, 0, 0, 0]
freq = normalized_freq(text)
pprint(freq)
print('sum of frequencies: %d' % sum(freq))

array([0.33333333, 0.16666667, 0.25      , 0.08333333, 0.08333333,
       0.08333333, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        ])
sum of frequencies: 1


index of maximum likelyhood를 계산하는 함수 `maximum_likelyhood`를 구현하자.

In [4]:
monogram_prob = np.array([
    0.082, 0.015, 0.028, 0.043, 0.127, 0.022, 0.020, 0.061, # ABCDEFGH
    0.070, 0.002, 0.008, 0.040, 0.024, 0.067, 0.075, 0.019, # IJKLMNOP
    0.001, 0.060, 0.063, 0.091, 0.028, 0.010, 0.023, 0.001, # QRSTUVWX
    0.020, 0.001                                            # YZ
])

def maximum_likelyhood(text):
    return -normalized_freq(text) * np.log2(monogram_prob)

# some code for testing
maximum_likelyhood(text)

array([1.20274409, 1.00981561, 1.28960734, 0.37829329, 0.24809163,
       0.45886272, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        ])

이를 구현하였으면 주된 코드인, `crack_hill`을 구현하자.

In [5]:
def crack_hill(ciphertext, blocksize):
    assert len(ciphertext) % blocksize == 0

    blocks = np.array_split(ciphertext, len(ciphertext) // blocksize)
    d = np.zeros((len(blocks), blocksize))
    for t in range(blocksize):
        for i in range(len(blocks)):
            d[i][t] = sum(blocks[i][:t + 1]) % 26
    
    inv_K = np.zeros((len(blocks), len(blocks)))
    I = np.full((blocksize,), -np.inf)
    p = np.zeros((len(blocksize),))
    iml